In [1]:
import numpy as np
from algorithms import conjugate_gradient, secant, Finite_Difference, armijo
from cost_functions import V_a, gradV_a, V_b, gradV_b
from numpy.linalg import norm, eig
from functools import partial


In [4]:
def steepest_descent(x0,
                     cost_function,
                     gradient_function = None,
                     step_size = 'armijo',
                     threshold = 1e-4, 
                     log = False, 
                     h = 1e-8, 
                     max_iter = 1e6, 
                     gamma = 1.5, 
                     r = 0.8, 
                     fd_method = 'central', 
                     track_history = False):
    '''
    Performs vanilla gradient descent. 
    Args: 
        x0 :: np.array
            Initial point of minization. Shape (n,)
        cost_function :: Python function
            Cost function to minimize. Rn -> R. 
        gradient_function :: Python function, or None
            Gradient of cost_function. Rn -> Rn
            If None, finite difference estimation of gradient is used. 
        step_size :: float or String
            Step size to use during gradient descent. 
            If 'armijo', Armijo step size selection is used. 
            Default: 'armijo
        threshold :: float
            Threshold at which to stop minimization. Values 
            should be close to 0. Default: 1e-4
        log :: bool
            True to log optimization progress. Default: False
        h :: float
            Parameter for finite difference estimation. 
            Default 1e-8
        max_iter :: int
            Maximum optimization iterations. Default: 1e6
        gamma :: float
            Gamma parameter for armijo. Default is 1.5. 
        r :: float
            r parameter for armijo. Default is 0.8. 
        fd_method :: string
            Method for finite difference estimation. 
            Options: 'central', 'forward'
        track_history :: bool
            True to track points visited and corresponding cost. 
    Returns: 
        x :: np.array
            Point at which minimization is reached. Shape (n,)
        minimum :: float
            Value of cost function at optimizer. 
        x_history :: list
            List of points visisted. (if track_history = True)
        V_history :: list
            List of costs visisted. (if track_history = True)
    '''

    #if no gradient function available, use finite difference appx
    if gradient_function == None: 
        fd = Finite_Difference(cost_function, fd_method, h)
        gradient_function = fd.estimate_gradient

    x_history, V_history = [],[]
    #initialize iterator, x, and gradient
    i = 0
    x = x0
    gradient = gradient_function(x)
    minimum = cost_function(x)

    #iterate until near zero gradient or max iterations reached
    while norm(gradient) >= threshold and i <= max_iter: 

        #update gradient
        gradient = gradient_function(x)

        #determine step size
        if step_size == 'armijo':
            step_size = armijo(x, cost_function, gradient, gamma, r, log = False)
        elif isinstance(step_size, (int, float)):
            pass
        else: 
            raise ValueError('step size should be float, int or "armijo"')
        
        # move to a new x by moving from the original x in the negative
        # direction of the gradient according to a given step size
        x = x - step_size*gradient
        minimum = cost_function(x)

        #result tracking
        i += 1
        if log and i % 1e4 == 0: 
            print(f'x = {x}, V(x) = {minimum:.5f}')
        if track_history: 
            x_history.append(x), V_history.append(minimum)

    if track_history:
        return x, minimum, x_history, V_history
    else: 
        return x, minimum



In [6]:
def augmented_lagrangian(x0, 
                         cost_function, 
                         gradient_function, 
                         equality_constraints = None, 
                         inequality_constraints = None,
                         threshold = 1e-8, 
                         sigma_max = 1e8):

    def phi(x):
        cost = cost_function(x)
        lambda_eq = lambd[:num_ec , :]
        lambda_ineq = lambd[num_ec:num_c , :]
        sigma_eq = sigma[:num_ec , :]
        sigma_ineq = sigma[num_ec:num_c , :]
    
        for constraint in equality_constraints:
            ec = constraint(x)
            cost -= lambda_eq * ec + 0.5 * sigma_eq * ec **2
        for i,constraint in enumerate(inequality_constraints):
            ic = constraint(x)
            if ic <= sigma_ineq[i] / sigma_ineq[i]: 
                t = -lambda_ineq[i] * ic + 0.5 * sigma_ineq[i] * ic**2 
            else:
                t = -0.5 * lambda_ineq[i]**2 / sigma_ineq[i] 
            cost += t
        return(cost)

    x_history, V_history = [],[]
    num_ec = len(equality_constraints)
    num_ic = len(inequality_constraints)
    num_c = num_ec + num_ic

    lambd = np.zeros((num_c,1))
    sigma = np.ones((num_c,1))

    cost = 1e12 * sigma
    x = x0

    while norm(cost) > threshold: 

        print(phi(x))

        x,_ = steepest_descent(x, phi, None, step_size = 1e-4, threshold = 1e-3,)
        previous_cost = cost
        inequality_cost = [cost(x) for cost in equality_constraints]
        equality_cost = [cost(x) for cost in inequality_constraints]
        cost = np.array(inequality_cost + equality_cost).reshape((-1,1))

        if any(sigma >= sigma_max):
            break
        if norm(cost, np.inf) > 0.25 * norm(previous_cost, np.inf):
            for i in range(num_c):
                if np.abs(cost[i]) > 0.25 * norm(previous_cost, np.inf):
                    sigma[i] *= 10
            continue
        lambd = lambd - (sigma * cost)
    minimum = cost_function(x)
    return x, minimum
    

In [7]:



v = lambda x: -x[0, 0] * x[1, 0]
h1 = lambda x: -x[0, 0] - x[1, 0]**2 + 1
h2 = lambda x: x[0, 0] + x[1, 0]

cost_function = v
inequality_constraints = [h1, h2]
equality_constraints = []
gradient_function = None
x0 = np.array([[10], [1]])

x, minimum = augmented_lagrangian(x0,
                    cost_function, 
                    gradient_function,
                    equality_constraints, 
                    inequality_constraints)
x, minimum 

[40.]
[-1.25049956]


KeyboardInterrupt: 

In [16]:
for x in []:
    print(11)